## Optimizing Newspaper and Magazine Shelf Space and Pricing for Maximum Profit

### Decision Variables

* $x_A, x_B, x_m \in \Z_+$ : indicating the number of Newspaper $A$, Newspaper $B$, and Magazines purchased.
* $y_A, y_B, y_m, y_p \in \Z_+$ : indicating the number of Newspaper $A$ sold individually, Newspaper $B$ sold, Magazines sold individually, and “bundle” (newspaper $A$ + magazine) units sold.
* $z_A, z_B \in \Z_+ $ : indicating the number of unsold/returned copies of Newspaper $A$ and $B$.

### Parameters

* $c_A, c_B, c_m$ : Purchase (cost) prices for $A,B$ and magazine, respectively. (B is cheaper)
* $p_A​,p_B​,p_m​$ : Selling prices (per unit) for $A, B$ and magazine, respectively.
* $r_A, r_B$ : Refunds (return values) for unsold copies of $A$ and $B$.
    - $r_A =  0.40 c_A$
    - $r_B = 0.10 c_B$ 
* $d_A, d_B, d_m, d_p$ :  Demands (maximum potential sales) for $A$, $B$, magazine, and the bundle, respectively.
* $C$: Total Budget Available
* $S_n, S_m$: Shelf capacities for newspapers and magazines, respectively.
* Discount: 8% off $(p_A+p_m)$ for each bundle sale, so bundle revenue is $0.92 (pA+pm)$.

### Constraints

1. Budget Constraints

    $c_A x_A + c_B x_B + c_m x_m \leq C$

2. Shelf-Space Constraints

    * Newspapers (both $A$ and $B$ share the same shelf):

        $x_A +  x_B  \leq S_n$
    
    * Magazines have their own shelf:
    
        $ x_m \leq S_m$

3. Demand Constraints:

    $y_A \leq d_A$, <br>
    $y_B \leq d_B$, <br>
    $y_m \leq d_m$, <br>
    $y_p \leq d_p$, <br>

4. Purchase‐to‐Sale (and Returns) Balances

    * Newspaper $B$

        $y_B +  z_B  = x_B$
    
    * Newspaper $A$:
    
        $y_A + y_p + z_A  = x_A$ (the bundle $y_p$ also uses up an A)

    * Magazine (assuming no returns, just leftover is wasted or not allowed):
    
        $y_m + y_p  \leq x_m$

5. Nonnegativity

    $x_A​,x_B​,x_m​,y_A​,y_B​,y_m​,y_p​,z_A​,z_B​ \geq 0$


    



### Objective Function

Maximize total profit, computed as:

* Revenue from individual newspaper and magazine sales:

    $p_A​ y_A ​+ p_B​ y_B ​+ p_m​ y_m​$

* Revenue from bundle sales at 8% discount:

    $0.92 (p_A + p_m) y_p​$

* Refund for unsold units of $A$ and $B$:

    $r_A ​z_A ​+ r_B ​z_B$

* Minus the total purchase cost:

    $−c_A ​x_A​ − c_B​ x_B​ − c_m​ x_m​​$


Putting it all together:

\begin{equation}

max \ \ p_A​ y_A ​+ p_B​ y_B ​+ p_m​ y_m + 0.92 (p_A + p_m) y_p + r_A ​z_A ​+ r_B ​z_B −c_A ​x_A​ − c_B​ x_B​ − c_m​ x_m​

\end{equation}


In [1]:
import cplex 
from cplex.exceptions import CplexError
from docplex.mp.model import Model
import time
import pandas as pd
import numpy as np

In [2]:
m = Model('newsvendor_shelf', log_output=True)

Defining Decision Variables

In [3]:
xA = m.continuous_var(name='xA', lb=0)
xB = m.continuous_var(name='xB', lb=0)
xm = m.continuous_var(name='xm', lb=0)
yA = m.continuous_var(name='yA', lb=0)
yB = m.continuous_var(name='yB', lb=0)
ym = m.continuous_var(name='ym', lb=0)
yp = m.continuous_var(name='yp', lb=0)
zA = m.continuous_var(name='zA', lb=0)
zB = m.continuous_var(name='zB', lb=0)


Example Parameter Values

In [4]:
# budget
C = 300

# costs
cA = 5
cB = 4.5
cm = 3

# selling prices
pA = 8
pB = 8
pm = 5

# demand
dA = 20
dB = 20
dm = 15
dp = 10

# shelf capacities
Sn = 30
Sm = 20

# return values
rA = 0.40 * cA
rB = 0.10 * cB

# pack discount
pp = 0.92 * (pA + pm)


Defining Constraints

In [5]:
# budget constraint
m.add_constraint(xA * cA + xB * cB + xm * cm <= C)

# shelf capacity constraints
m.add_constraint(xA + xB <= Sn)
m.add_constraint(xm <= Sm)

# demand constraints
m.add_constraint(yA <= dA)
m.add_constraint(yB <= dB)
m.add_constraint(ym <= dm)
m.add_constraint(yp <= dp)

# purchase to sale balances 
m.add_constraint(yA + yp + zA == xA)
m.add_constraint(yB + zB == xB)
m.add_constraint(ym + yp <= xm)


docplex.mp.LinearConstraint[](ym+yp,LE,xm)

In [6]:
obj_fn = pA * yA + pB * yB + pm * ym + 0.92 * (pA + pm) * yp + rA * zA + rB * zB - cA * xA - cB * xB - cm * xm
m.set_objective('max', obj_fn)

In [7]:
m.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 2 columns.
Aggregator did 2 substitutions.
Reduced LP has 2 rows, 5 columns, and 6 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           139.600000


docplex.mp.solution.SolveSolution(obj=134.8,values={xA:10,xB:20,xm:20,yA..

In [8]:
m.solve_status

<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [9]:
#print model
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: newsvendor_shelf

Maximize
 obj: - 5 xA - 4.500000000000 xB - 3 xm + 8 yA + 8 yB + 5 ym
      + 11.960000000000 yp + 2 zA + 0.450000000000 zB
Subject To
 c1: 5 xA + 4.500000000000 xB + 3 xm <= 300
 c2: xA + xB <= 30
 c3: xm <= 20
 c4: yA <= 20
 c5: yB <= 20
 c6: ym <= 15
 c7: yp <= 10
 c8: yA + yp + zA - xA = 0
 c9: yB + zB - xB = 0
 c10: ym + yp - xm <= 0

Bounds
End



In [10]:
# objective value
print(m.objective_value)

134.8


In [11]:
# decision variables values
print(m.solution.get_value(xA))
print(m.solution.get_value(xB))
print(m.solution.get_value(xm))
print(m.solution.get_value(yA))
print(m.solution.get_value(yB))
print(m.solution.get_value(ym))
print(m.solution.get_value(yp))
print(m.solution.get_value(zA))
print(m.solution.get_value(zB))

10.0
20.0
20.0
5.0
20.0
15.0
5.0
0
0


## Considering The Stochastic Context

Now, since the deterministic linear model is completed. We will model the problem as two-stage decision making process

### Solving the Newsvendor problem via Deterministic Equivalent Problem:

Assume we know the random variables: $D_A, D_B, D_p, D_m$ that describes the behavior of the demands: $d_A, d_B, d_p, d_m$

#### First Stage Problem

$
\begin{array}{rl}
\min_x & c^Tx - \mathbb{E}[R(x, D)] ={\color{red}\mathbb{E}[cx-R(x, D)]}\\
\text{s.t} & x_A + x_B \leq S_n\\
& x_m \leq S_m\\
& c^Tx \leq C\\
& x \geq 0
\end{array}
$

Let the vectors be: 

$
\begin{array}{rcl}
c & = & (c_A, c_B, c_m); \\
x & = & (x_A, x_B, x_m); \\
D & = & (D_A, D_B, D_m, D_p); \\
\end{array}
$

#### Second Stage Problem

$
\begin{array}{rrl}
R(x,d) = &\max_{y,z} & py + rz\\
& \text{s.t} & y\leq d\\
 && y_A+y_p+z_A \leq x_A \\
 && y_B+z_B \leq x_B \\
 && y_p + y_m \leq x_m \\
 && y,z\geq 0
\end{array}
$

Let the vectors be: 

$
\begin{array}{rcl}
z & = & (z_A, z_B); \\
y & = & (y_A, y_B, y_m, y_p); \\
p & = & (p_A, p_B, p_m, 0.92(p_A+p_m)); \\
r & = & (r_A, r_B); \\
\end{array}
$

Transforming the second stage in minimizing...

#### Second Stage Problem
$\begin{array}{rrrl}
Q(x,d)=&-R(x,d) = &\min_{y,z} & -py - rz\\
&& \text{s.t} & y\leq d\\
    &&& y_A+y_p+z_A \leq x_A \\
    &&& y_B+z_B \leq x_B \\
    &&& y_p + y_m \leq x_m \\
    &&& y,z\geq 0
\end{array}
$

#### Deterministic Equivalent Problem
$\begin{array}{rl}
\min_{x, y_{wi}, z_{wi}} & \mathbb{E}[c^Tx - py_{wi} - rz_{wi}] = {\color{red}\sum_i \mathbb{P}(w_i)[c^Tx - py_{w_i} - rz_{w_i}]} = {\color{red}\sum_i \frac{1}{n}[c^Tx - py_{w_i} - rz_{w_i}]}\\
\text{s.t} & x_A + x_B \leq S_n\\
& x_m \leq S_m\\
& c^Tx \leq C\\
& y_{wi} \leq d_{wi}\\
& y_{A_{wi}}+y_{p_{wi}}+z_{A_{wi}} \leq x_A \\
& y_{B_{wi}}+z_{B_{wi}} \leq x_B \\
& y_{p_{wi}} + y_{m_{wi}} \leq x_m \\
& x,y,z\geq 0 \\
\end{array}
$

In [12]:
# Load the data from the Excel file
data = pd.read_excel('./samples.xlsx')

In [13]:
m_demand = Model('newsvendor_shelf', log_output=True)

In [14]:
data.sample(5)

,newspaper1,newspaper2,magazine,pack
207,14,26,107,9
607,14,32,29,19
66,16,46,22,6
114,12,31,80,16
827,17,17,46,8


### Parameters

Example Parameter Values

In [ ]:
# budget
C = 4000

# costs
cA = 5
cB = 0.9 * cA
cm = 8

# selling prices
pA = 15
pB = 0.9 * pA
pm = 20

# getting demands
dA = data['newspaper1']
dB = data['newspaper2']
dm = data['magazine']
dp = data['pack']

# shelf capacities
Sn = 150
Sm = 100

# return values
rA = 0.40 * cA
rB = 0.10 * cB

# pack discount
pp = 0.92 * (pA + pm)


In [16]:
p = [pA, pB, pm, pp]
r = [rA, rB]
c = [cA, cB, cm]
d = [dA, dB, dm, dp]
S = [Sn, Sm]

### Decision variables

In [17]:
x = m_demand.continuous_var_list(3, lb=0, name='x') #xA, xB, xm
y = m_demand.continuous_var_matrix(4, len(data.index), name="y") #yA, yB, ym, yp for each sample demand
z = m_demand.continuous_var_matrix(2, len(data.index), name="z") #zA, zB


### Constraints

In [18]:
# shelf capacity constraints
m_demand.add_constraint(sum(x[i] for i in range(0,2)) <= Sn)
m_demand.add_constraint(x[2] <= Sm)

# budget constraint
m_demand.add_constraint(sum(x[i] * c[i] for i in range(len(c))) <= C)

# demand constraints
for i in range(len(data.index)):
    m_demand.add_constraint(y[0,i] <= dA[i])
    m_demand.add_constraint(y[1,i] <= dB[i])
    m_demand.add_constraint(y[2,i] <= dm[i])
    m_demand.add_constraint(y[3,i] <= dp[i])

# purchase to sale balances
for i in range(len(data.index)):
    m_demand.add_constraint(y[0,i] + y[3,i] + z[0,i] <= x[0])
    m_demand.add_constraint(y[1,i] + z[1,i] <= x[1])
    m_demand.add_constraint(y[2,i] + y[3,i] <= x[2])



### Objective Function

In [19]:
# objective function
obj_fn = (1/len(data.index)) *  sum(c[i] * x[i] for i in range(len(c))) - \
                                sum(p[j] * y[j,i] for j in range(4) for i in range(len(data.index))) - \
                                sum(r[j] * z[j,i] for j in range(2) for i in range(len(data.index)))
          

In [20]:
obj_fn

docplex.mp.LinearExpr(0.005x_0+0.004x_1+0.008x_2-15y_0_0-15y_0_1-15y_0_2-15y_0_3-15y_0_4-15y_0_5-15y_0_6-15y_0_7-15y_0_8-15y_0_9-15y_0_10-15y_0_11-15y_0_12-15y_0_13-15y_0_14-15y_0_15-15y_0_16-15y_0_17-15y_0_18-15y_0_19-15y_0_20-15y_0_21-15y_0_22-15y_0_23-15y_0_24-15y_0_25-15y_0_26-15y_0_27-15y_0_28-15y_0_29-15y_0_30-15y_0_31-15y_0_32-15y_0_33-15y_0_34-15y_0_35-15y_0_36-15y_0_37-15y_0_38-15y_0_39-15y_0_40-15y_0_41-15y_0_42-15y_0_43-15y_0_44-15y_0_45-15y_0_46-15y_0_47-15y_0_48-15y_0_49-15y_0_50-15y_0_51-15y_0_52-15y_0_53-15y_0_54-15y_0_55-15y_0_56-15y_0_57-15y_0_58-15y_0_59-15y_0_60-15y_0_61-15y_0_62-15y_0_63-15y_0_64-15y_0_65-15y_0_66-15y_0_67-15y_0_68-15y_0_69-15y_0_70-15y_0_71-15y_0_72-15y_0_73-15y_0_74-15y_0_75-15y_0_76-15y_0_77-15y_0_78-15y_0_79-15y_0_80-15y_0_81-15y_0_82-15y_0_83-15y_0_84-15y_0_85-15y_0_86-15y_0_87-15y_0_88-15y_0_89-15y_0_90-15y_0_91-15y_0_92-15y_0_93-15y_0_94-15y_0_95-15y_0_96-15y_0_97-15y_0_98-15y_0_99-15y_0_100-15y_0_101-15y_0_102-15y_0_103-15y_0_104-15y_0_105-1

In [21]:
m_demand.set_objective('min', obj_fn)
m_demand.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 16 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 14 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 4987 rows and 986 columns.
Reduced LP has 2016 rows, 5017 columns, and 7032 nonzeros.
Presolve time = 0.01 sec. (7.73 ticks)
Symmetry aggregator did 4453 additional substitutions.

Iteration log . . .
Iteration:     1   Scaled dual infeas =          2399.995000
Iteration:    62   Scaled dual infeas =          2040.799977
Iteration:   131   Scaled dual infeas =          1700.798959
Iteration:   259   Scaled dual infeas =          1091.999958
Iteration:   447   Scaled dual infeas =           625.999958
Iteration:   635   Scaled dual infeas =            92.000000
Iteration:   661   Dual objective     =      -2392681.050000

Dual crossover.
 

docplex.mp.solution.SolveSolution(obj=-2.29168e+06,values={x_0:93,x_1:57..

In [23]:
print(m_demand.solve_status)

JobSolveStatus.OPTIMAL_SOLUTION


In [24]:
# objective value
print(m_demand.objective_value)

-2291679.307


In [25]:
# decision variables values
for i in range(len(data.index)):
    print(f"Sample {i+1}:")
    print("xA:", m_demand.solution.get_value(x[0]))
    print("xB:", m_demand.solution.get_value(x[1]))
    print("xm:", m_demand.solution.get_value(x[2]))
    print("yA:", m_demand.solution.get_value(y[0,i]))
    print("yB:", m_demand.solution.get_value(y[1,i]))
    print("ym:", m_demand.solution.get_value(y[2,i]))
    print("yp:", m_demand.solution.get_value(y[3,i]))
    print("zA:", m_demand.solution.get_value(z[0,i]))
    print("zB:", m_demand.solution.get_value(z[1,i]))
    print()

Sample 1:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 13.0
yB: 53.0
ym: 95.0
yp: 5.0
zA: 75.0
zB: 4.0

Sample 2:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 45.0
yB: 17.0
ym: 30.0
yp: 16.0
zA: 32.0
zB: 40.0

Sample 3:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 22.0
yB: 38.0
ym: 53.0
yp: 14.0
zA: 57.0
zB: 19.0

Sample 4:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 18.0
yB: 33.0
ym: 89.0
yp: 11.0
zA: 64.0
zB: 24.0

Sample 5:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 36.0
yB: 9.0
ym: 26.0
yp: 14.0
zA: 43.0
zB: 48.0

Sample 6:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 13.0
yB: 29.0
ym: 23.0
yp: 13.0
zA: 67.0
zB: 28.0

Sample 7:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 10.0
yB: 42.0
ym: 65.0
yp: 8.0
zA: 75.0
zB: 15.0

Sample 8:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 26.0
yB: 47.0
ym: 16.0
yp: 10.0
zA: 57.0
zB: 10.0

Sample 9:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 7.0
yB: 29.0
ym: 81.0
yp: 19.0
zA: 67.0
zB: 28.0

Sample 10:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 8.0
yB: 57.0
ym: 84.0
yp: 4.0
zA: 81.0
zB: 0

Sample 11:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 8.0
yB: 47.0
ym: 95.0
yp: 5.0
zA: 80

In [ ]:
# Save the results to an Excel file
results = {
    'xA': [m_demand.solution.get_value(x[0])] * len(data),
    'xB': [m_demand.solution.get_value(x[1])] * len(data),
    'xm': [m_demand.solution.get_value(x[2])] * len(data),
    'yA': [m_demand.solution.get_value(y[0,i]) for i in range(len(data.index))],
    'yB': [m_demand.solution.get_value(y[1,i]) for i in range(len(data.index))],
    'ym': [m_demand.solution.get_value(y[2,i]) for i in range(len(data.index))],
    'yp': [m_demand.solution.get_value(y[3,i]) for i in range(len(data.index))],
    'zA': [m_demand.solution.get_value(z[0,i]) for i in range(len(data.index))],
    'zB': [m_demand.solution.get_value(z[1,i]) for i in range(len(data.index))]
}
results_df = pd.DataFrame(results)

In [27]:
results_df.to_excel('results.xlsx', index=False)
# Print the results
print("Results saved to results.xlsx")

Results saved to results.xlsx


### Wait and See solution (WS)

In [28]:
def get_ws_sample(i):
    print(f"Sample {i+1}:")
    print("xA:", m_demand.solution.get_value(x[0]))
    print("xB:", m_demand.solution.get_value(x[1]))
    print("xm:", m_demand.solution.get_value(x[2]))
    print("yA:", m_demand.solution.get_value(y[0,i]))
    print("yB:", m_demand.solution.get_value(y[1,i]))
    print("ym:", m_demand.solution.get_value(y[2,i]))
    print("yp:", m_demand.solution.get_value(y[3,i]))
    print("zA:", m_demand.solution.get_value(z[0,i]))
    print("zB:", m_demand.solution.get_value(z[1,i]))
    print()

get_ws_sample(3)



Sample 4:
xA: 93.0
xB: 57.0
xm: 100.0
yA: 18.0
yB: 33.0
ym: 89.0
yp: 11.0
zA: 64.0
zB: 24.0



### Perfect Information Distribution (PID)

For calculating PID, it will be necessary to code the second stage problem

In [ ]:
def R(x, d):
    m_resource = Model('second_stage_resource', log_output=True)
    y = m_resource.continuous_var(4, name="y") #yA, yB, ym, yp for each sample demand
    z = m_resource.continuous_var(2, name="z") #zA, zB

    for i in range(0,4):
        m_resource.add_constraint(y[i] <= d[i])
        m_resource.add_constraint(y[0] + y[3] + z[0] <= x[0])
        m_resource.add_constraint(y[1] + z[1] <= x[1])
        m_resource.add_constraint(y[2] + y[3] <= x[2])
        

    return obj.getValue(), y.X, z.X

In [30]:
PID = pd.Series([R(x[i].X, tab.loc[i,'Sample'])[0] - c*x[i].X for i in tab.index])

NameError: name 'tab' is not defined

In [ ]:
(y[0] , d[0])

KeyError: 0

In [ ]:
# demand constraints
for i in range(0,4):
    m_demand.add_constraint(y[i] <= d[i])


# purchase to sale balances
for i in range(len(data.index)):
    m_demand.add_constraint(y[0,i] + y[3,i] + z[0,i] <= x[0])
    m_demand.add_constraint(y[1,i] + z[1,i] <= x[1])
    m_demand.add_constraint(y[2,i] + y[3,i] <= x[2])

